In [3]:
# Python native
import os

os.chdir("/home/tim/Development/OCPPM/")
import pickle
import random
from copy import copy
from datetime import datetime
from statistics import median as median
from sys import platform
from typing import Any, Callable

# Data handling
import numpy as np
import pandas as pd
import ocpa.algo.predictive_monitoring.factory as feature_factory

# PyG
import torch

# PyTorch TensorBoard support
import torch.utils.tensorboard

# Object centric process mining
from ocpa.algo.predictive_monitoring.obj import Feature_Storage as FeatureStorage

# # Simple machine learning models, procedure tools, and evaluation metrics
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.tensorboard.writer import SummaryWriter
from torch_geometric.loader import DataLoader
from tqdm import tqdm

# Custom imports
# from loan_application_experiment.feature_encodings.efg.efg import EFG
from loan_application_experiment.feature_encodings.efg.efg_sg import EFG_SG

# from importing_ocel import build_feature_storage, load_ocel, pickle_feature_storage
from loan_application_experiment.models.geometric_models import AdamsGCN, GraphModel, AGNN
from test_gnn_on_efg_sg import evaluate_best_model, load_datasets, prepare_dataloaders, configure_adams_model,print_dataset_summaries

In [4]:
state_dict_path = "models/runs/AGNN_20230713_21h55m"

bpi17_config = {
    "STORAGE_PATH": "data/BPI17/feature_encodings/EFG/efg",
    "SPLIT_FEATURE_STORAGE_FILE": "BPI_split_[C2_P2_P3_P5_O3_Action_EventOrigin_OrgResource].fs",
    "TARGET_LABEL": (feature_factory.EVENT_REMAINING_TIME, ()),
    "SUBGRAPH_SIZE": 4,
    "BATCH_SIZE": 64,
    "RANDOM_SEED": 42,
    "EPOCHS": 30,
    "early_stopping": 7,
    "optimizer_settings": {
        "lr": 0.001,
        "betas": (0.9, 0.999),
        "eps": 1e-08,
        "weight_decay": 0,
        "amsgrad": False,
    },
    "loss_fn": torch.nn.L1Loss(),
    "verbose": True,
    "skip_cache": False,
}


def seed_worker(worker_id: int) -> None:
    # worker_seed = torch.initial_seed() % RANDOM_SEED
    worker_seed = bpi17_config["RANDOM_SEED"]
    np.random.seed(worker_seed)
    random.seed(worker_seed)


generator = torch.Generator().manual_seed(bpi17_config["RANDOM_SEED"])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Get data and dataloaders
ds_train, ds_val, ds_test = load_datasets(
    bpi17_config["STORAGE_PATH"],
    bpi17_config["SPLIT_FEATURE_STORAGE_FILE"],
    bpi17_config["TARGET_LABEL"],
    bpi17_config["SUBGRAPH_SIZE"],
    train=True,
    val=True,
    test=True,
)
# print_dataset_summaries(ds_train, ds_val, ds_test)
train_loader, val_loader, test_loader = prepare_dataloaders(
    batch_size=bpi17_config["BATCH_SIZE"],
    ds_train=ds_train,
    ds_val=ds_val,
    ds_test=ds_test,
    seed_worker=seed_worker,
    generator=generator,
)

In [6]:
# Set model configuration
model = AGNN(hidden_channels=128, out_channels=1)
model.to(device)
just_get_preds = lambda a, b: (a, b)
normalized_evaluation_dict = evaluate_best_model(
    model=model,
    model_state_dir=state_dict_path,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    metric=torch.nn.L1Loss(),
    device=torch.device("cpu"),
    verbose=bpi17_config["verbose"],
)

RuntimeError: Error(s) in loading state_dict for AGNN:
	Missing key(s) in state_dict: "conv1.att", "conv1.bias", "conv1.lin_l.weight", "conv1.lin_l.bias", "conv1.lin_r.weight", "conv1.lin_r.bias", "conv2.att", "conv2.bias", "conv2.lin_l.weight", "conv2.lin_l.bias", "conv2.lin_r.weight", "conv2.lin_r.bias". 
	Unexpected key(s) in state_dict: "conv1.lin_msg.weight", "conv1.lin_msg.bias", "conv1.lin_self.weight", "conv1.lin_self.bias", "conv2.lin_msg.weight", "conv2.lin_msg.bias", "conv2.lin_self.weight", "conv2.lin_self.bias". 

In [22]:
train_preds = normalized_evaluation_dict[list(normalized_evaluation_dict.keys())[0]][0]
train_preds.mean()

tensor(0.)

In [38]:
b0=next(iter(train_loader))
def eval_batch(batch, model):
    batch_inputs, batch_adjacency_matrix, batch_labels = (
        batch.x.float(),
        batch.edge_index,
        batch.y.float(),
    )
    return model(batch_inputs, batch_adjacency_matrix), batch_labels
pd.DataFrame(b0.x).describe()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
count,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,...,256.000000,256.000000,256.000000,256.00000,256.000000,256.000000,256.000000,256.00000,256.000000,256.000000
mean,-0.064036,-0.023751,-0.058362,0.108887,0.220502,-0.067831,0.039286,0.033040,-0.107734,0.065532,...,-0.038242,-0.024978,-0.003184,-0.01388,-0.189062,-0.055596,0.007124,175868.53125,135573.343750,8739.429688
std,0.894721,0.952523,0.641111,1.153635,1.118303,0.000000,1.062570,1.035689,0.435029,1.065598,...,0.743240,0.000000,0.000000,0.00000,0.823200,0.651811,0.794690,80577.12500,14970.163086,15889.636719
min,-0.294600,-0.234576,-0.098432,-0.294600,-0.525150,-0.067831,-0.285884,-0.398522,-0.134923,-0.329228,...,-0.084695,-0.024978,-0.003184,-0.01388,-0.709011,-0.162898,-1.161597,31509.00000,114852.000000,557.000000
25%,-0.294600,-0.234576,-0.098432,-0.294600,-0.525150,-0.067831,-0.285884,-0.398522,-0.134923,-0.329228,...,-0.084695,-0.024978,-0.003184,-0.01388,-0.698642,-0.162898,-0.057565,86364.00000,128226.000000,2521.000000
50%,-0.294600,-0.234576,-0.098432,-0.294600,-0.525150,-0.067831,-0.285884,-0.398522,-0.134923,-0.329228,...,-0.084695,-0.024978,-0.003184,-0.01388,-0.608030,-0.162898,-0.057565,234196.00000,128226.000000,3832.000000
75%,-0.294600,-0.234576,-0.098432,-0.294600,1.891142,-0.067831,-0.285884,-0.398522,-0.134923,-0.329228,...,-0.084695,-0.024978,-0.003184,-0.01388,-0.118018,-0.162898,-0.057565,234196.00000,150853.000000,5061.000000
max,3.394430,4.263014,10.159348,3.394430,1.891142,-0.067831,3.497917,2.508842,6.825533,2.930726,...,11.807139,-0.024978,-0.003184,-0.01388,2.441271,7.360688,3.254531,234196.00000,150853.000000,58953.000000


In [1]:
2e-3

0.002